# Ranking school examination results using multivariate hierarchical models

In [44]:
import numpy as np
import scipy.stats as sp
import matplotlib.pyplot as plt

In [45]:
import json 
with open("data/data.json") as file:
    data = json.load(file)
    
with open("data/init.json") as file:
    init = json.load(file)
    
with open("data/params.json") as file:
    params = json.load(file)

In [46]:
for idx, item in data.items():
    data[idx] = np.array(item).reshape(-1)

for idx, item in init.items():
    init[idx] = np.array(item).reshape(-1)
    
for idx, item in params.items():
    params[idx] = np.array(item).reshape(-1)

In [47]:
data.keys(), init.keys(), params.keys()

(dict_keys(['Gender', 'LRT', 'M', 'N', 'R', 'school', 'School_denom', 'School_gender', 'Y', 'VR']),
 dict_keys(['theta', 'phi', 'gamma', 'beta', 'Omega', 'alpha']),
 dict_keys(['R_shape', 'School_denom_shape', 'School_gender_shape', 'Venom_shape', 'Omega_shape', 'alpha_shape']))

In [73]:
params

{'R_shape': array([3, 3]),
 'School_denom_shape': array([1978,    3]),
 'School_gender_shape': array([1978,    2]),
 'Venom_shape': array([1978,    3]),
 'Omega_shape': array([3, 3]),
 'alpha_shape': array([38,  3])}

In [48]:
hyperparams = dict()
hyperparams["tau0"] = 10**(-3)

In [70]:
def sampler(n, data, init, params, hyperparams):
    
    # initialisation de la chaine 
    chain = dict()
    for key, param in init.items():
        chain[key] = np.zeros((n+1, param.shape[0]))
        chain[key][0, :] = param
    
    # définition des constantes
    LRT_sum = np.sum(data["LRT"])
    N   = data["LRT"].shape[0]
    mu  = np.zeros(data["Y"].shape) ## à supprimer après le calcul de mu
    tau = np.zeros(data["Y"].shape) ## à supprimer après le calcul de tau
    tau0 = hyperparams["tau0"]
    n_beta = init["beta"].shape[0]
    n_schools = np.max(data["school"])
    
    ### TODO: définir v
    ### TODO: définir u
    
    # mise à jour des paramètres
    for i in range(1, n+1):
        ### TODO: calcul de mu_ij
        ### TODO: calcul de tau_ij
        
        # mise à jour de phi: mcmc 
        # phi peut prendre toutes les valeurs réelles, 
        # donc on utilise le "random walk M.-H. scheme"
        theta_current = chain["theta"][i-1, :]
        phi_current = chain["phi"][i-1, :]
        
        phi_proposal = phi_current + np.random.normal()
        
        phi_top = (-tau0/2) * np.power(phi_proposal, 2)
        phi_top += (LRT_sum/2) * phi_proposal
        phi_top -= (np.exp(theta_current)/2) * np.sum(
            np.exp(phi_proposal*data["LRT"]) * np.power(data["Y"]-mu, 2)
        )
    
        phi_bottom = (-tau0/2) * np.power(phi_current, 2)
        phi_bottom += (LRT_sum/2) * phi_current
        phi_bottom -= (np.exp(theta_current)/2) * np.sum(
            np.exp(phi_current*data["LRT"]) * np.power(data["Y"]-mu, 2)
        )
        
        phi_acceptance = np.exp(phi_top-phi_bottom)
        if phi_acceptance < np.random.rand():
            chain["phi"][i,:] = phi_proposal
            phi_current = phi_proposal
        else:
            chain["phi"][i,:] = phi_current

        # mise à jour de theta: mcmc 
        # theta peut prendre toutes les valeurs réelles, 
        # donc on utilise le "random walk M.-H. scheme"        
        theta_proposal = theta_current + np.random.normal()
        
        theta_top = (-tau0/2) * np.power(theta_proposal, 2)
        theta_top += (N/2) * theta_proposal
        theta_top -= (np.exp(theta_proposal)/2) * np.sum(
            np.exp(phi_current*data["LRT"]) * np.power(data["Y"]-mu, 2)
        )
        
        theta_bottom = (-tau0/2) * np.power(theta_current, 2)
        theta_bottom += (LRT_sum/2) * theta_current
        theta_bottom -= (np.exp(theta_current)/2) * np.sum(
            np.exp(phi_current*data["LRT"]) * np.power(data["Y"]-mu, 2)
        ) 
        
        theta_acceptance = np.exp(theta_top-theta_bottom)
        if theta_acceptance < np.random.rand():
            chain["theta"][i,:] = theta_proposal
            theta_current = theta_proposal
        else:
            chain["theta"][i,:] = theta_current
                      
        # mise à jour de beta: normale multivariée
        alpha_current = chain["alpha"][i-1, :]
        beta_current = chain["beta"][i-1, :]
        
        beta_cov_inv = tau0 * np.eye(n_beta) 
        for k in range(N):
            beta_cov_inv = beta_cov_inv + tau[k] * np.dot(v[[k], :], v[[k], :].T)
        beta_cov = np.linalg.inv(beta_cov_inv)
        
        beta_mean = tau * Y * v.T 
        for j in range(1, n_schools+1):
            school_j = (data["school"] ==j)
            alpha_j = alpha_current[(3*(j-1)):(3*j)]
            beta_mean = beta_mean - np.dot(alpha_j.T, np.dot(u[school_j, :].T, v[school_j, :]))
        beta_mean = np.dot(beta_cov, beta_mean)
        beta_proposal = random.Generator.multivariate_normal(beta_mean, beta_cov)
        
        chain["beta"][i, :] = beta_proposal
        beta_current = beta_proposal
        
        # mise à jour de T: mcmc
        
        
        # mise à jour de gamma: normale
        
        
        # mise à jour de alpha: normale
        
    
    return chain

In [71]:
np.array([2314,123]).T


array([2314,  123])

In [72]:
sampler(10, data, init, params, hyperparams)

/tmp/ipykernel_5298/3187798468.py:67: RuntimeWarning: overflow encountered in exp
  theta_acceptance = np.exp(theta_top-theta_bottom)


NameError: name 'v' is not defined